In [28]:
from dotenv import load_dotenv
import os

In [29]:
load_dotenv()


True

In [30]:
hf_token = os.getenv("hf_token")
model_id = "sentence-transformers/all-MiniLM-L6-v2"


In [38]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
<All keys matched successfully>


torch.Size([4, 4])


In [39]:
model.encode(sentences)

array([[ 0.04123014,  0.06826037, -0.00916981, ...,  0.03575852,
         0.0154005 , -0.00475998],
       [ 0.0195328 ,  0.02978142, -0.0168549 , ...,  0.02254233,
        -0.01423946,  0.00686363],
       [ 0.03636552,  0.08672781, -0.00809686, ...,  0.02911964,
         0.00933333, -0.02031948],
       [-0.00026413, -0.00924167, -0.01055697, ...,  0.00430718,
        -0.02677994, -0.01220293]], dtype=float32)

## Import and Clean Data

In [40]:
import pandas as pd
import matplotlib.pyplot as plt


In [41]:
raw_data = pd.read_csv("../data/Salary.csv")
education_levels = ["high school", "bachelor degree", "masters degree", "phd"]
raw_data

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000.0,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000.0,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000.0,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000.0,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000.0,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000.0,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000.0,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000.0,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000.0,China,Korean,0


In [45]:
df = raw_data[["Job Title","Education Level","Years of Experience","Country","Salary"]]
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [46]:
isomapping = {"usa":"us", "china":"cn","australia":"au","uk":"uk","canada":"ca"}
df['country'] = df['country'].str.lower().map(isomapping)

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_5556\1632140083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country'] = df['country'].str.lower().map(isomapping)


In [47]:
df

,job_title,education_level,years_of_experience,country,salary
0,Software Engineer,1,5.0,uk,90000.0
1,Data Analyst,2,3.0,us,65000.0
2,Manager,3,15.0,ca,150000.0
3,Sales Associate,1,7.0,us,60000.0
4,Director,2,20.0,us,200000.0
...,...,...,...,...,...
6679,Director of Marketing,3,20.0,uk,200000.0
6680,Sales Associate,0,3.0,au,50000.0
6681,Financial Manager,1,4.0,cn,55000.0
6682,Marketing Manager,2,14.0,cn,140000.0


In [48]:
sampled_df = df.sample(n=1000, replace=True, random_state=42)


## Setting up Lin Reg Model with word embeddings

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
features = ["job_title","education_level", "years_of_experience", "country"]